In [19]:
from pathlib import Path
import numpy as np
import pandas as pd
import json

from DataAnalysisClass import *
from clustering_work import *
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns

single = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV5Z', 'aug-cc-pV6Z']
single_polarized = ['aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ']
double = ['d-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z']
double_polarized = ['d-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
all_basis_sets = single + single_polarized + double + double_polarized

In [21]:

data_base=Path("/home/ahurta92/data/may2024")
mol_list_path = data_base.joinpath("mols_list")
# read mol_list

with open(mol_list_path) as mol_list:
    full_mol_list = mol_list.read().splitlines()

# split into N lists

def split_list(lst, n):
    k, m = divmod(len(lst), n)
    return (lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


In [48]:
N=4
mol_list = list(split_list(full_mol_list, N))


for i in range(N):
    print(' '.join(mol_list[i]))

FNO CH3SH N2H2 CH3NH2 HOCl SiO HBS HBO CH2BH PH3O ClF HF BH2Cl SiH4 SF2 P2H4 NaCN OCl2
CH3Cl SCl2 HCCF LiH BF SiH3F HCONH2 CS S2H2 FCN NaCl SiH3Cl N2H4 NH2Cl LiCl NH3O NH2OH BH3
SO2 HCN CH3BH2 NaLi HOF CH3F HNS CH3OH O3 CH2NH ClCN Mg2 NH3 H2O PH3 BHF2 LiCN NH2F
HCl HCCCl HCOOH HCHS LiH_s HCHO SH2 CSO HOOH CO HCP NOCl HNO NaH OF2 HNC BH2F LiBH4


In [36]:
def query_beta_data(df,omega_b,omega_c):
    om=df.Bfreq.unique()

    b = om[omega_b]
    c = om[omega_c]

    return df.query('Bfreq==@b and Cfreq==@c')
def query_alpha_data(df,omega):
    om=df.omega.unique()
    b = om[omega]
    return df.query('omega==@b')
def process_beta_df(beta_df):
    beta_df.Afreq = beta_df.Afreq.round(3)
    beta_df.Bfreq = beta_df.Bfreq.round(3)
    beta_df.Cfreq = beta_df.Cfreq.round(3)
    beta_df['ijk'] = beta_df['A'].astype(str) + beta_df['B'].astype(str) + beta_df['C'].astype(str)
    beta_df = beta_df.drop(columns=['A', 'B', 'C'])
    return beta_df


class mra_data:
    def __init__(self,mol,data_base):
        self.mol = mol
        self.data_dir = data_base.joinpath(mol)
        calc_path = self.data_dir.joinpath("calc_path.json")
        self.calc_path_json = json.load(open(calc_path))
        self.alpha=pd.DataFrame(json.load(open(self.data_dir.joinpath(self.calc_path_json['alpha_json_path']))))
        self.alpha['basis']='MRA'
        self.alpha['mol']=mol
        self.beta=process_beta_df(pd.DataFrame(json.load(open(self.data_dir.joinpath(self.calc_path_json['quadratic_json_path'])))))
        self.beta['basis']='MRA'
        self.beta['molecule']=mol

        self.moldft_calc_info=json.load(open(self.data_dir.joinpath(self.calc_path_json['moldft']['outfiles']['calc_info'])))
        self.energy=self.moldft_calc_info['return_energy']


    def get_beta(self,omega_b,omega_c):
        return query_beta_data(self.beta,omega_b,omega_c)
    def get_alpha(self,omega):
        return query_alpha_data(self.alpha,omega)
        


,Afreq,Bfreq,Cfreq,Beta,ijk,basis,molecule
378,-0.074,0.037,0.037,-6.169006,XXX,MRA,FNO
379,-0.074,0.037,0.037,-10.651349,XXY,MRA,FNO
380,-0.074,0.037,0.037,0.000009,XXZ,MRA,FNO
381,-0.074,0.037,0.037,-10.651349,XYX,MRA,FNO
382,-0.074,0.037,0.037,-2.843129,XYY,MRA,FNO
383,-0.074,0.037,0.037,-0.000011,XYZ,MRA,FNO
384,-0.074,0.037,0.037,0.000009,XZX,MRA,FNO
385,-0.074,0.037,0.037,-0.000011,XZY,MRA,FNO
386,-0.074,0.037,0.037,2.396506,XZZ,MRA,FNO
387,-0.074,0.037,0.037,-10.723635,YXX,MRA,FNO


In [47]:
mol='CO'
output_data=data_base.joinpath("output")
mra_data(mol,output_data).get_beta(0,1)

,Afreq,Bfreq,Cfreq,Beta,ijk,basis,molecule
27,-0.02,0.0,0.02,-0.875541,XXX,MRA,CO
28,-0.02,0.0,0.02,-0.359797,XXY,MRA,CO
29,-0.02,0.0,0.02,1.576057,XXZ,MRA,CO
30,-0.02,0.0,0.02,-0.359890,XYX,MRA,CO
31,-0.02,0.0,0.02,0.193599,XYY,MRA,CO
32,-0.02,0.0,0.02,0.052139,XYZ,MRA,CO
33,-0.02,0.0,0.02,1.546903,XZX,MRA,CO
34,-0.02,0.0,0.02,0.052752,XZY,MRA,CO
35,-0.02,0.0,0.02,-0.700880,XZZ,MRA,CO
36,-0.02,0.0,0.02,-0.190104,YXX,MRA,CO


In [34]:
data={}
output_data=data_base.joinpath("output")
available_data = []
not_available_data = []

for mol in full_mol_list:
    try:
        data[mol]=mra_data(mol,output_data)
        available_data.append(mol)
    except FileNotFoundError as e:
        not_available_data.append(mol)
    except:
        data[mol]=None

In [7]:
print(len(available_data))
print(available_data)

34
['FNO', 'CH3SH', 'N2H2', 'CH3NH2', 'HOCl', 'SiO', 'LiH', 'N2H4', 'NH2Cl', 'LiCl', 'NH3O', 'NH2OH', 'BH3', 'SO2', 'HCN', 'CH3BH2', 'HOF', 'CH3F', 'CH3OH', 'O3', 'CH2NH', 'ClCN', 'NH3', 'H2O', 'PH3', 'BHF2', 'LiH_s', 'HCHO', 'SH2', 'CSO', 'HOOH', 'CO', 'HCP', 'HNO']


In [8]:
len(not_available_data)

38

In [11]:
data['FNO'].get_beta(4,4)

,Afreq,Bfreq,Cfreq,Beta,ijk,basis,mol
378,-0.074,0.037,0.037,-6.169006,XXX,MRA,FNO
379,-0.074,0.037,0.037,-10.651349,XXY,MRA,FNO
380,-0.074,0.037,0.037,0.000009,XXZ,MRA,FNO
381,-0.074,0.037,0.037,-10.651349,XYX,MRA,FNO
382,-0.074,0.037,0.037,-2.843129,XYY,MRA,FNO
383,-0.074,0.037,0.037,-0.000011,XYZ,MRA,FNO
384,-0.074,0.037,0.037,0.000009,XZX,MRA,FNO
385,-0.074,0.037,0.037,-0.000011,XZY,MRA,FNO
386,-0.074,0.037,0.037,2.396506,XZZ,MRA,FNO
387,-0.074,0.037,0.037,-10.723635,YXX,MRA,FNO


In [15]:
data['FNO'].get_alpha(4)

,omega,ij,alpha
36,0.037071,XX,2.200960e+01
37,0.037071,XY,4.119187e+00
38,0.037071,XZ,1.285936e-07
39,0.037071,YX,4.118654e+00
40,0.037071,YY,1.556308e+01
41,0.037071,YZ,-1.037627e-06
42,0.037071,ZX,1.868251e-06
43,0.037071,ZY,2.066425e-06
44,0.037071,ZZ,1.083903e+01
